## gensim으로 네이버 기사 토픽 모델링 해보기

> 토픽 모델링을 적용하기 위해 텍스트를 처리합니다.

> 토픽 모델링 라이브러리인 gensim을 사용해봅니다.

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [1]:
from tqdm import tqdm_notebook # progress bar
from konlpy.tag import Mecab #Mecab, Okt 등 형태소 분석기 불러오기
import numpy as np
import string # 특수문자
import re
import warnings # 경고 알림 제거
import pickle
from gensim import corpora # lda를 돌릴 수 있는 형태로 바꿔줌
from gensim import models # 여기에 lda 모델이 들어있다
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시합니다.

### 2. 텍스트 전처리 함수 만들기

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    for i in string.punctuation:
        SW.add(i)

    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path) as f:
        for word in f:
            SW.add(word)

    return SW
SW = define_stopwords("stopwords-ko.txt")

UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 0: illegal multibyte sequence

In [9]:
SW

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 b'\xea\xb0\x80\n',
 b'\xea\xb0\x80\xea\xb9\x8c\xec\x8a\xa4\xeb\xa1\x9c\n',
 b'\xea\xb0\x80\xeb\xa0\xb9\n',
 b'\xea\xb0\x81\n',
 b'\xea\xb0\x81\xea\xb0\x81\n',
 b'\xea\xb0\x81\xec\x9e\x90\n',
 b'\xea\xb0\x81\xec\xa2\x85\n',
 b'\xea\xb0\x96\xea\xb3\xa0\xeb\xa7\x90\xed\x95\x98\xec\x9e\x90\xeb\xa9\xb4\n',
 b'\xea\xb0\x99\xeb\x8b\xa4\n',
 b'\xea\xb0\x99\xec\x9d\xb4\n',
 b'\xea\xb0\x9c\xec\x9d\x98\xec\xb9\x98\xec\x95\x8a\xea\xb3\xa0\n',
 b'\xea\xb1\xb0\xeb\x8b\x88\xec\x99\x80\n',
 b'\xea\xb1\xb0\xeb\xb0\x94\n',
 b'\xea\xb1\xb0\xec\x9d\x98\n',
 b'\xea\xb2\x83\n',
 b'\xea\xb2\x83\xea\xb3\xbc \xea\xb0\x99\xec\x9d\xb4\n',
 b'\xea\xb2\x83\xeb\x93\xa4\n',
 b'\xea\xb2\x8c\xeb\x8b\xa4\xea\xb0\x80\n',
 b'\xea\xb2\x8c\xec\x9a\xb0\xeb\x8b\xa4\n',
 b'\xea\xb2\xa8\xec\x9a\xb0\n',
 b'\xea\xb2\xac\xec\xa7\x80\xec\x97\x90\xec\x84\x9c\n',
 b

In [2]:
def read_documents(input_file_name):
    """문서들을 주어진 이름의 파일로부터 읽어들여 돌려준다."""
    
    corpus = []
    
    with open(input_file_name, 'rb') as f:
        temp_corpus = pickle.load(f)
        
    for page in temp_corpus:
        corpus += page
    
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수.
    for doc in docs:
        doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)

    return docs

def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    for i in string.punctuation:
        SW.add(i)

    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path) as f:
        for word in f:
            SW.add(word)

    return SW


def text_tokenizing(corpus, tokenizer):
    
    mecab = Mecab()
    token_corpus = []
    

    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
                
            token_corpus.append(token_text)
            
    elif tokenized == "morph":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)

    elif tokenizer == "word":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
        

    return token_corpus


input_file_name = "data/naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "word"

문서 읽기의 과정은 앞서 단어 임베딩의 경우와 다르지 않다. 다음 과정은 문서-단어 행렬을 만드는 과정이다.

In [3]:
print(tokenized_text[0])

['본문', '내용', '플레이어', '플레이어', '오류', '우회', '함수', '추가', '유명', '스타트업', '투자자', '밸류', '성인', '대상', '직무', '교육', '서비스', '오프라인', '제공', '러닝', '스푼', '대표', '이창민', '사모', '투자', '방식', '규모', '투자', '유치', '이번', '라운드', '프리', '시리즈', '단계', '유명', '스타트업', '투자자', '다양', '전문가', '참여', '투자', '포스트', '밸류', '러닝', '스푼', '모토', '성장', '필요', '순간', '러닝', '스푼', '러닝', '스푼', '교육', '범위', '파이낸스', '비즈니스', '데이터', '사이언스', '부동산', '데이', '클래스', '다양', '온라인', '교육', '오프라인', '심층', '교육', '서비스', '제공', '삼성증권', '화증', '화학', '기업', '대상', '사내', '교육', '진행', '러닝', '스푼', '올해', '상반기', '파이썬', '활용', '데이터', '분석', '시각', '파이썬', '활용', '금융', '데이터', '수집', '분석', '디지털', '트랜스', '포메이션', '기반', '융합', '과정', '국내', '유수', '기업', '제공', '설명', '동안', '러닝', '스푼', '파이낸스', '영역', '시작', '카테고리', '확장', '러닝', '스푼', '이번', '투자', '채용', '연계', '전일제', '교육', '사업', '시도', '예정', '자금', '올해', '하반기', '스타트업', '채용', '연계', '부트', '캠프', '코스', '사용', '예정', '부트', '캠프', '코스', '평일', '파이썬', '데이터', '분석', '언어', '공부', '주말', '데이터', '분석가', '퍼포먼스', '각자', '커리어', '패스', '전공', '심화', '과정', '제공', '예정', '이창민', '러닝',

### 3. 토픽 모델링에 사용할 함수들 확인하기

In [4]:
# 문서-단어 행렬 만들기
# 어휘(vocabulary) 학습
dictionary = corpora.Dictionary(tokenized_text)
# 문서-단어 행렬(document-term matrix) 생성
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [5]:
print(dictionary)

Dictionary(327 unique tokens: ['각자', '강사진', '거리', '경제', '공부']...)


In [6]:
corpus[0][:5]

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1)]

In [7]:
# TFIDF 문서-단어 행렬 생성
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[0][:5]

[(0, 0.06232726104668814),
 (1, 0.06232726104668814),
 (2, 0.06232726104668814),
 (3, 0.0764315702870735),
 (4, 0.06232726104668814)]

In [8]:
model = models.ldamodel.LdaModel(corpus, num_topics=4, id2word=dictionary)

In [9]:
model.show_topic(3, 10)

[('에듀', 0.03548237),
 ('블록', 0.03131921),
 ('교육', 0.028588695),
 ('패키지', 0.02640994),
 ('초급', 0.024832888),
 ('코딩', 0.020167248),
 ('출시', 0.016783467),
 ('지니', 0.015812188),
 ('초등학생', 0.013613544),
 ('가능', 0.013063768)]

### 4. 토픽 모델링을 추가하여 코드 완성하기

In [10]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 3

NUM_TOPIC_WORDS = 30


def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary


def print_topic_words(model):

    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

# document-term matrix를 만들고,
corpus, dictionary = build_doc_term_mat(tokenized_text)
# LDA를 실행.
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
# 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	교육	0.020064108073711395
	에듀	0.019615206867456436
	블록	0.016438454389572144
	플로	0.014306404627859592
	초급	0.014013572596013546
	패키지	0.012968584895133972
	출시	0.012588164769113064
	텐서	0.010947481729090214
	제공	0.010392452590167522
	내용	0.010076988488435745
	활용	0.009872214868664742
	본문	0.00967254489660263
	파이썬	0.00937303900718689
	가능	0.009322361089289188
	코딩	0.008824671618640423
	플레이어	0.008590282872319221
	러닝	0.008431630209088326
	지니	0.007824854925274849
	모델	0.007635747082531452
	사용	0.007484680041670799
	접근	0.007203388027846813
	초등학생	0.006948885507881641
	데이터	0.006911374628543854
	지원	0.006664160173386335
	성능	0.0066604153253138065
	스푼	0.006648664362728596
	구글	0.006617039907723665
	학습	0.006384249776601791
	기업	0.006313104182481766
	고급	0.006254277192056179


Topic ID: 1
	블록	0.029162965714931488
	에듀	0.028093811124563217
	교육	0.024360517039895058
	지니	0.02031286619603634
	패키지	0.01944926381111145
	초급	0.01651754230260849
	출시	0.014452628

### 5. pyLDAvis를 통한 토픽 모델링 결과 시각화하기

In [11]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1     -0.009479  0.0       1        1  56.606251
0      0.008340  0.0       2        1  36.207973
2      0.001139  0.0       3        1   7.185773, topic_info=     Category       Freq  Term      Total  loglift  logprob
term                                                       
6     Default  21.000000    교육  21.000000  30.0000  30.0000
243   Default  16.000000   패키지  16.000000  29.0000  29.0000
235   Default  23.000000    에듀  23.000000  28.0000  28.0000
240   Default  15.000000    초급  15.000000  27.0000  27.0000
242   Default  11.000000    코딩  11.000000  26.0000  26.0000
200   Default  13.000000    출시  13.000000  25.0000  25.0000
238   Default  14.000000    지니  14.000000  24.0000  24.0000
15    Default   9.000000    내용   9.000000  23.0000  23.0000
215   Default   8.000000    플로   8.000000  22.0000  22.0000
94    Default   5.000000    추가   5.000000  21.0000  21.0000
88    Default  10.000000    제공  10.000000  20.0000  20.0000
38    Default   9.000000    본문   9.000000  19.0000  19.0000
241   Default   8.000000  초등학생   8.000000  18.0000  18.0000
121   Default   9.000000    활용   9.000000  17.0000  17.0000
205   Default   6.000000    텐서   6.000000  16.0000  16.0000
114   Default   9.000000  플레이어   9.000000  15.0000  15.0000
233   Default   5.000000   솔루션   5.000000  14.0000  14.0000
59    Default   6.000000    스푼   6.000000  13.0000  13.0000
28    Default   7.000000    러닝   7.000000  12.0000  12.0000
117   Default   4.000000    함수   4.000000  11.0000  11.0000
14    Default   5.000000    기자   5.000000  10.0000  10.0000
47    Default   7.000000    사용   7.000000   9.0000   9.0000
229   Default  22.000000    블록  22.000000   8.0000   8.0000
237   Default   6.000000    이해   6.000000   7.0000   7.0000
17    Default   6.000000    뉴스   6.000000   6.0000   6.0000
227   Default   5.000000    기술   5.000000   5.0000   5.0000
236   Default   6.000000    원리   6.000000   4.0000   4.0000
143   Default   4.000000    모델   4.000000   3.0000   3.0000
71    Default   5.000000    오류   5.000000   2.0000   2.0000
106   Default   8.000000   파이썬   8.000000   1.0000   1.0000
...       ...        ...   ...        ...      ...      ...
239    Topic3   0.384541    체계   4.484433   0.1767  -5.1999
225    Topic3   0.382287    교재   4.515101   0.1641  -5.2058
243    Topic3   1.263004   패키지  16.497009   0.0634  -4.0107
124    Topic3   0.347741    개발   4.162158   0.1507  -5.3005
14     Topic3   0.469322    기자   5.803033   0.1182  -5.0007
241    Topic3   0.627280  초등학생   8.046102   0.0815  -4.7106
15     Topic3   0.758356    내용   9.996368   0.0542  -4.5208
240    Topic3   1.083851    초급  15.075123   0.0005  -4.1637
6      Topic3   1.411241    교육  21.833988  -0.1059  -3.8998
88     Topic3   0.714165    제공  10.170259  -0.0230  -4.5809
200    Topic3   0.874370    출시  13.231211  -0.0838  -4.3785
38     Topic3   0.666642    본문   9.476774  -0.0213  -4.6497
71     Topic3   0.408876    오류   5.215187   0.0871  -5.1386
235    Topic3   1.300224    에듀  23.615191  -0.2663  -3.9817
17     Topic3   0.456538    뉴스   6.130174   0.0358  -5.0283
121    Topic3   0.635060    활용   9.605784  -0.0833  -4.6983
28     Topic3   0.509378    러닝   7.265156  -0.0246  -4.9188
114    Topic3   0.629177  플레이어   9.902903  -0.1231  -4.7076
227    Topic3   0.442660    기술   5.996337   0.0270  -5.0592
237    Topic3   0.482273    이해   6.818366  -0.0158  -4.9735
238    Topic3   0.784145    지니  14.685770  -0.2970  -4.4874
205    Topic3   0.477459    텐서   6.881001  -0.0350  -4.9835
47     Topic3   0.501082    사용   7.509643  -0.0741  -4.9352
215    Topic3   0.519929    플로   8.146604  -0.1186  -4.8983
236    Topic3   0.459348    원리   6.908772  -0.0777  -5.0222
75     Topic3   0.389717    우회   5.160333   0.0497  -5.1866
229    Topic3   0.806502    블록  22.592791  -0.6996  -4.4593
223    Topic3   0.497161    가능   9.358310  -0.3020  -4.9431
106    Topic3   0.466290   파이썬   8.407930  -0.2591  